In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

In [5]:
def find_longest_falling_streak(file_path, start_date=None, end_date=None, green_candles_limit=5):
    # Загрузка данных из CSV файла
    df = pd.read_csv(file_path, parse_dates=['Date'])
    
    # Фильтрация по заданным датам
    if start_date:
        df = df[df['Date'] >= pd.to_datetime(start_date)]
        
    if end_date:
        df = df[df['Date'] <= pd.to_datetime(end_date)]

    # Добавляем колонку для определения падения (True, если свеча красная, иначе False)
    df['is_red'] = df['Close'] < df['Open']
    
    # Колонка для небольших зеленых свечей (по изменению цены)
    df['small_green'] = (df['Close'] > df['Open']) & (df['Close'] - df['Open'] < 0.01 * df['Open'])  # Задать порог для "небольшой свечи"
    
    # Вспомогательные переменные
    max_streak = 0
    current_streak = 0
    green_candles_in_streak = 0
    streak_start_index = None
    max_streak_info = {"start_date": None, "end_date": None, "length": 0}

    for i, row in df.iterrows():
        if row['is_red'] or (row['small_green'] and green_candles_in_streak < green_candles_limit):
            # Начало новой последовательности падения
            if current_streak == 0 and row['is_red']:
                streak_start_index = i

            if current_streak > 0 or row['is_red']:  # Продолжение только если началась с красной свечи
                current_streak += 1
                if row['small_green']:
                    green_candles_in_streak += 1
        else:
            # Проверяем максимальную последовательность, если она начинается и заканчивается красной свечой
            if current_streak > 0 and df.loc[i - 1, 'is_red']:
                if current_streak > max_streak:
                    max_streak = current_streak
                    max_streak_info['start_date'] = df.loc[streak_start_index, 'Date']
                    max_streak_info['end_date'] = df.loc[i - 1, 'Date']
                    max_streak_info['length'] = max_streak

            current_streak = 0
            green_candles_in_streak = 0

    # Если последняя последовательность оказалась самой длинной
    if current_streak > 0 and df.loc[i, 'is_red']:
        if current_streak > max_streak:
            max_streak_info['start_date'] = df.loc[streak_start_index, 'Date']
            max_streak_info['end_date'] = df.iloc[-1]['Date']
            max_streak_info['length'] = current_streak

    # Вывод результатов
    print(f"Самая длинная последовательность падений: {max_streak_info['length']} свечей")
    print(f"Начало: {max_streak_info['start_date']}, Конец: {max_streak_info['end_date']}")
    
    # Визуализация графика свечей с выделенным участком
    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                         open=df['Open'],
                                         high=df['High'],
                                         low=df['Low'],
                                         close=df['Close'])])
    
    # Добавляем красный фон на период падения
    if max_streak_info['start_date'] and max_streak_info['end_date']:
        fig.add_vrect(
            x0=max_streak_info['start_date'], x1=max_streak_info['end_date'],
            fillcolor="red", opacity=0.3,
            layer="below", line_width=0)

    fig.update_layout(
        title="График актива с максимальным падением",
        xaxis_title="Дата",
        yaxis_title="Цена",
        xaxis_rangeslider_visible=False)
    
    fig.show()

In [6]:
# Пример использования:
file_path = "/Users/mchomak/Projects/RL_for_Trading/data/raw/PEPE_per_hour_1_2024_10_13.csv"
find_longest_falling_streak(file_path, start_date= "2024-04-13", end_date= "2024-10-12", green_candles_limit= 5)

Самая длинная последовательность падений: 21 свечей
Начало: 2024-06-29 08:00:00, Конец: 2024-06-30 04:00:00
